# Setup

### Acquire Data

In [1]:
# pip install ir_datasets
import ir_datasets
dataset = ir_datasets.load("clinicaltrials/2021")

In [23]:
# for doc in dataset.docs_iter():
#     doc # namedtuple<doc_id, title, condition, summary, detailed_description, eligibility>

### Install ES package, connect to ES service

In [5]:
# !pip install elasticsearch==7.9.1
# !pip install elasticsearch==7.17.3

In [2]:
from elasticsearch import Elasticsearch
es = Elasticsearch(HOST='http://localhost', PORT='9200')

# Load Data into ElasticSearch

In [3]:
#define mapping to store data
mapping = {
    "settings":
    {
        "index":
        {
            "number_of_shards":1,
            "number_of_replicas":1
        }
    },
    "mappings":
    {
        "properties":
        {
            "content":
            {
                "type":"text",
                "fielddata":True,
                "term_vector":"with_positions_offsets_payloads",
                "store":True,
                "analyzer":"whitespace"
            }
        }
    }
}


In [19]:
#Delete any previously created index named 'ir3'
es.indices.delete(index='ir3', ignore=[404, 400])

#Create index with the mapping defined
response_createIndex = es.indices.create(index='ir3', body=mapping)
print("Create Index Response:",response_createIndex)

Create Index Response: {'acknowledged': True, 'shards_acknowledged': True, 'index': 'ir3'}


In [ ]:
#Load data into the index

id = 0
for doc in dataset.docs_iter():
    es.index('ir3', id=id, body=doc)
    id+=1